In [9]:
import pickle
import torch
from src.model import Transformer, TransformerConfig
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [27]:
with open("models/config.json", "r") as f:
    settings = json.loads(f.read())

with open("models/word_data.json", "r") as f:
    word_data = json.loads(f.read())

word_to_index = {k:int(v) for k, v in word_data["word_to_index"].items()}
index_to_word = {int(k):v for k, v in word_data["index_to_word"].items()}

In [50]:
config = TransformerConfig(vocab_size=settings["vocabulary_size"], max_input_length=settings["max_input_length"], num_heads=settings["num_heads"], num_blocks=settings["num_blocks"], embedding_size=settings["embedding_size"])
transformer = Transformer(config)
transformer.load_state_dict(torch.load("models/model.pt", map_location=torch.device(device)))
transformer.to(device)
transformer.train()
print("Model loaded")

Model loaded


In [51]:
def generate_next_token(tokens=None, sample=False):
    if tokens == None:
        tokens = [len(index_to_word)-1]
    x = torch.LongTensor([tokens]).to(device)
    with torch.no_grad():
        y = transformer(x)
    # Don't allow the model to generate <unknown> tokens
    #y = y[:, :, :y.shape[2]-1]
    y = torch.nn.functional.softmax(y[:, -1, :y.shape[2]-1].view(-1).detach().cpu(), dim=0)
    if not sample:
        pred = y.argmax(dim=-1).view(-1)
    else:
        dist = torch.distributions.categorical.Categorical(probs=y)
        pred = dist.sample([1])
    next_word = pred.item()
    return next_word

def print_sentence(words):
    print(" ".join([index_to_word[word] for word in words]))

def generate_sentence(start=None):
    if start == None:
        sentence = []
    else:
        words = start.split(" ")
        sentence = [word_to_index[x] for x in words]
    
    while len(sentence) < settings["max_input_length"]:
        next_word = generate_next_token(sentence)
        sentence += [next_word]
    
    print_sentence(sentence)


In [52]:
generate_sentence("if thou didst")
generate_sentence("you shall not pass")
generate_sentence("and")
generate_sentence("and")
generate_sentence("and")
generate_sentence("i")
generate_sentence("i")
generate_sentence("i")

if thou didst not hear me speak . hamlet . i am a gentleman . i would not speak with you . horatio . you shall not have me ,
you shall not pass me . vernon . i will do you any slight harm . vernon . i humbly thank you . exit . marcellus . i am glad
and the painter . sicinius . i thank you , sir . menenius . well , well , be gone . exeunt scene iv . rome . a room seek
and i will help thee . exit . scene iii . london . a room in the tower . enter the king , salisbury , northumberland , northumberland , hotspur
and , in this great trial , let me be proclaimd . let me be bold , for i am so fearful that i have sick , and think no
i have a quarter of the world . falstaff . i would i had been a garter . prince . thou wilt , jack , thou wilt have my visor
i warrant you , sir , that you shall be out of a more virtuous than a great and so wise . i have seen you in person of the
i will not talk of a song . rosalind . what , ist ? rosalind . marry , heres a husband . rosalind . why , sir , i pray
